In [1]:
import pandas as pd

In [2]:
cd ../

/home/anna/PycharmProjects/DataJournalismApp


In [3]:
import util.data_preprocessing as dp
import util.keywords_extraction as ke
import util.topic_extraction as te
import util.ner_finder as nf

# Load and preprocess data

In [90]:
# data = pd.read_csv("data/raw_data/news33.csv", sep=";")
# data["topic"] = data["rubric"].apply(lambda t: t.split(", ")[-1].split(" ")[0])

# data = pd.read_csv("data/raw_data/news2020.csv")
# data.drop(["subrubric", "tags"], axis=1, inplace=True)

data = pd.read_csv("data/raw_data/news_lenta_99-19.csv")

data.head()

,tags,text,title,topic,url
0,Общество,Миллиардер Илон Маск в резкой форме ответил бр...,Илон Маск назвал педофилом спасавшего детей из...,Мир,https://lenta.ru/news/2018/07/16/su57/
1,Рынки,США и их западные союзники рассматривают возмо...,США задумались о распечатывании нефтяного резерва,Экономика,https://lenta.ru/news/2018/07/16/foes/
2,Преступность,Празднование победы сборной Франции на чемпион...,Празднование победы на ЧМ во Франции закончило...,Мир,https://lenta.ru/news/2018/07/15/fra_bezumie/
3,Политика,География использования лимузинов проекта «Кор...,Песков рассказал о планах на президентские лим...,Россия,https://lenta.ru/news/2018/07/16/delo_shyut/
4,Музыка,Американская поп-певица Бритни Спирс случайно ...,Грудь Бритни Спирс вновь выскочила из лифчика ...,Культура,https://lenta.ru/news/2018/07/16/pedomusk/


In [91]:
# lenta 99-18

data["date"] = pd.to_datetime(data["url"].apply(lambda x: ".".join(x.split("/")[4:7])))
data.drop(["tags", "url"], axis=1, inplace=True)
data = data[data["date"] >= pd.to_datetime("2015.01.01")]
data.reset_index(drop=True, inplace=True)
data.head()

,text,title,topic,date
0,Миллиардер Илон Маск в резкой форме ответил бр...,Илон Маск назвал педофилом спасавшего детей из...,Мир,2018-07-16
1,США и их западные союзники рассматривают возмо...,США задумались о распечатывании нефтяного резерва,Экономика,2018-07-16
2,Празднование победы сборной Франции на чемпион...,Празднование победы на ЧМ во Франции закончило...,Мир,2018-07-15
3,География использования лимузинов проекта «Кор...,Песков рассказал о планах на президентские лим...,Россия,2018-07-16
4,Американская поп-певица Бритни Спирс случайно ...,Грудь Бритни Спирс вновь выскочила из лифчика ...,Культура,2018-07-16


In [59]:
# news2020

data["source"].unique()

lenta_data = data[data["source"] == 'lenta.ru']
lenta_data["date"] = pd.to_datetime(lenta_data["publication_date"]).values

ria_data = data[data["source"] == 'ria.ru']
ria_data["date"] = pd.to_datetime(ria_data["publication_date"].apply(lambda x: " ".join(x.split(" ")[:2]))).values

name_to_num = {"янв": "01", "фев": "02", "мар": "03", "апр": "04", "мая": "05", "июн": "06", 
               "июл": "07", "авг": "08", "сен": "09", "окт": "10", "ноя": "11", "дек": "12"}
meduza_data = data[data["source"] == 'meduza.io']
meduza_data["date"] = meduza_data["publication_date"].apply(lambda x: x.split(", ")[1].split(" "))
meduza_data["date"] = meduza_data["date"].apply(lambda x: ".".join([x[0], name_to_num[x[1][:3]], x[2]]))


array(['lenta.ru', 'meduza.io', 'ria.ru', 'tjournal.ru'], dtype=object)

In [78]:
tjournal_data = data[data["source"] == 'tjournal.ru']
tjournal_data["_time"] = tjournal_data["publication_date"].astype("int64")
tjournal_data.head()

/home/anna/anaconda3/envs/journalism_env/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,source,title,text,publication_date,rubric,_time
20803,tjournal.ru,\n ФБК показал филь...,\n\n \n \n \n \n \n...,1598875929,NaN,1598875929
20804,tjournal.ru,\n На улицах Минска...,\n\n \n \n \n \n \n...,1598795150,NaN,1598795150
20805,tjournal.ru,\n В Черногории про...,\n\n \n \n \n \n \n...,1598821788,NaN,1598821788
20806,tjournal.ru,\n Путин и Лукашенк...,\n\n \n \n \n \n \n...,1598784941,NaN,1598784941
20807,tjournal.ru,\n МВД Белоруссии з...,\n\n \n \n \n \n \n...,1598801069,NaN,1598801069


In [76]:
tjournal_data.loc[20803, "text"]

'\n\n    \n    \n        \n        \n        \n             \n    \n         Это первый ролик из фильмов-расследований о региональной политике Сибири. Он посвящён Новосибирску. \n     \n\n        \n        \n            \n                                 \n                                             \n                        \n \n     \n\n         \n\n         \n         104 \n\n        \n             комментария \n\n             \n \n                    \n                     \n                    \n \n\n     \n    \n             В закладки \n    \n     \n \n\n                     \n                                            \n \n          Слушать \n \n                    \n                     \n                    \n \n\n     \n\n        \n             \n\n                 \n             \n\n        \n             \n\n                 \n             \n\n        \n             \n\n                 \n             \n\n        \n             \n\n                 \n             \n\n   

In [97]:
# preprocess text columns and add source and _time cols

# data = dp.preprocess_data(data.drop(["tags"], axis=1), ["title", "text"], "Новости 33", time_col="date")
data = dp.preprocess_data(data, ["title", "text"], "Lenta", time_col="date")

data.head()

,text,title,topic,date,source,_time
0,Миллиардер Илон Маск в резкой форме ответил бр...,Илон Маск назвал педофилом спасавшего детей из...,Мир,2018-07-16,Lenta,1531699200
1,США и их западные союзники рассматривают возмо...,США задумались о распечатывании нефтяного резерва,Экономика,2018-07-16,Lenta,1531699200
2,Празднование победы сборной Франции на чемпион...,Празднование победы на ЧМ во Франции закончило...,Мир,2018-07-15,Lenta,1531612800
3,География использования лимузинов проекта Корт...,Песков рассказал о планах на президентские лим...,Россия,2018-07-16,Lenta,1531699200
4,Американская поп-певица Бритни Спирс случайно ...,Грудь Бритни Спирс вновь выскочила из лифчика ...,Культура,2018-07-16,Lenta,1531699200


In [100]:
data["art_ind"] = data["date"].astype("str") + ": " + data["title"].apply(lambda x: x.replace("\n", " ").replace("\r", " "))

In [ ]:
# add existing kw and ne to news33 dataset, to save original text format


# kw_ne = pd.read_csv("data/raw_data/news33_kw_ne.csv", index_col=0)
# kw_ne["art_ind"] = kw_ne["date"] + ": " + kw_ne["title"]

# kw_ne.drop("text", axis=1, inplace=True)
# new_data = kw_ne.merge(data[["text", "art_ind"]], on="art_ind", how="inner")
# new_data.fillna("", inplace=True)
# new_data["text"] = new_data["text"].apply(lambda t: t.replace("\n", "\n\r"))
# new_data.to_parquet("news33_processed")

# Find keywords

In [ ]:
corpus = data["text"]
all_text_keywords, vectorizer = ke.get_keywords(corpus)

In [ ]:
data["keywords"] = all_text_keywords
data.head()

In [18]:
#ke.get_keywords_preprocessed(data["text"][:10], vectorizer)

['фапах; фап; медицинский; пункт; район; деятельность; района; фапов; пункта; прокуратура',
 'женщина; хрустальный; гусь; потерпевший; звонить; адвокаты; компенсация; звонок; неизвестный; кредит',
 'жириновский; губернатор; сипягин; собрание; лдпр; владимир; областной; обещать; лидер; регион',
 'теплов; составить; сетей; отопительный; м²; работа; подготовка; возле; вода; филиал',
 'скорость; физика; света; поток; полупроводниковый; квантовый; вычислительный; создание; самый; информация',
 'брат; старший; приговорить; строгий; признать; произойти; хотел; угар; телесный; судогде',
 'совет; территорий; генплан; архитектор; проект; застройка; областной; культурный; объект; города',
 'жириновский; региона; губернатор; владимир; торжество; телеграмма; сипягиным; предполагаться; правительственный; почтовый',
 'проверка; дорожный; хрустальный; гусь; автомобильный; прокурорский; мошенничество; выявить; контракт; деревня',
 'комбинат; тепличный; ленинский; крупный; виктор; приговор; размере; дох

# Find topics

In [20]:
topic_keywords, topics_pred, vectorizer, lda = te.get_topics(data["text"].values, topics=5, n_keywords=5)

In [21]:
topic_keywords

{0: ['владимирский', 'область', 'суд', 'владимир', 'дом'],
 1: ['владимирский', 'владимир', 'житель', 'администрация', 'область'],
 2: ['август', 'служба', 'владимирский', 'житель', 'дтп'],
 3: ['свой', 'владимирский', 'убийство', 'суд', 'летний'],
 4: ['владимирский', 'август', 'владимир', 'свой', 'рублей']}

In [24]:
topics = {0: "суд", 1: "социум", 2: "дтп", 3: "убийство", 4: "август"}

In [23]:
topics_pred

array([4, 2, 1, 1, 3, 3, 4, 1, 0, 2, 2, 1, 1, 3, 2, 2, 4, 2, 1, 2, 2, 0,
       4, 1, 3, 2, 1, 4, 4, 4, 1, 0, 1, 0, 1, 0, 0, 0, 2, 2, 0, 2, 0, 4,
       3, 1, 4, 1, 0, 3, 1, 4, 4, 0, 0, 4, 0, 1, 1, 0, 1, 1, 2, 0, 1, 0,
       0, 1, 4, 4, 4, 2, 1, 0, 0, 4, 1, 0, 0, 1, 2, 4, 4, 1, 4, 1, 4, 3,
       2, 4, 1, 1, 0, 4, 4, 1, 1, 3, 4, 1])

In [26]:
te.get_topics_pretrained(data["text"].values[:10], lda, vectorizer)

array([4, 2, 1, 1, 3, 3, 4, 1, 0, 2])

In [29]:
data = te.set_topics_to_data(data, topics_pred, topics)
data.head()

,date,rubric,title,text,_time,source,keywords,topic
index,,,,,,,,
0,2019-08-17,Новости 33; Общество,Прокуратура обнаружила ФАПы-призраки в Суздаль...,"Прокуратура обнаружила, что паспорт региональн...",1566000000,news33,фапах; фап; медицинский; пункт; район; деятель...,август
1,2019-08-17,Новости 33; Происшествия 33,В Гусь-Хрустальном женщина перевела мошенникам...,Следственный отдел полиции в Гусь-Хрустальном ...,1566000000,news33,женщина; хрустальный; гусь; потерпевший; звони...,дтп
2,2019-08-16,Новости 33; Общество,Жириновский: Мое имя тоже - Владимир. Всё сошл...,Во Владимирском академическом театре драмы про...,1565913600,news33,жириновский; губернатор; сипягин; собрание; лд...,социум
3,2019-08-16,Новости 33; Общество,Во Владимире трубопровод на улице Герцена убер...,Во Владимире модернизируют тепловую сеть на ул...,1565913600,news33,теплов; составить; сетей; отопительный; м²; ра...,социум
4,2019-08-16,Новости 33; Общество,Выпускник ВлГУ приблизил эру квантовых компьют...,Выпускник кафедры физики и прикладной математи...,1565913600,news33,скорость; физика; света; поток; полупроводнико...,убийство


# Find NE

In [36]:
data["ner"] = data["text"].apply(nf.finder)

In [37]:
data["ner"] = data["ner"].apply(lambda x: "; ".join([w.lower() for w in x]))
data.head()

,date,rubric,title,text,_time,source,keywords,topic,ner
index,,,,,,,,,
0,2019-08-17,Новости 33; Общество,Прокуратура обнаружила ФАПы-призраки в Суздаль...,"Прокуратура обнаружила, что паспорт региональн...",1566000000,news33,фапах; фап; медицинский; пункт; район; деятель...,август,суздальский; большое борисово; фапы район; лем...
1,2019-08-17,Новости 33; Происшествия 33,В Гусь-Хрустальном женщина перевела мошенникам...,Следственный отдел полиции в Гусь-Хрустальном ...,1566000000,news33,женщина; хрустальный; гусь; потерпевший; звони...,дтп,гусь-хрустальный; россия; мо; мвд; вгусе.ру; с...
2,2019-08-16,Новости 33; Общество,Жириновский: Мое имя тоже - Владимир. Всё сошл...,Во Владимирском академическом театре драмы про...,1565913600,news33,жириновский; губернатор; сипягин; собрание; лд...,социум,владимирская область; россия; цфо; владимирски...
3,2019-08-16,Новости 33; Общество,Во Владимире трубопровод на улице Герцена убер...,Во Владимире модернизируют тепловую сеть на ул...,1565913600,news33,теплов; составить; сетей; отопительный; м²; ра...,социум,во владимир; герцен; т плюс; владимирская тэц-2
4,2019-08-16,Новости 33; Общество,Выпускник ВлГУ приблизил эру квантовых компьют...,Выпускник кафедры физики и прикладной математи...,1565913600,news33,скорость; физика; света; поток; полупроводнико...,убийство,влгу; евгений седов
